### exercise


#### batch norm

题外话,出错的写法

**feed_dict里直接用的是tensor,其实应该先用sess.run把这个tensor结果运行出来结果,然后把这个结果赋值给feed_dict.**

出错的:
```python
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ###x_val=sess.run(x_value)
    feed_dict = {
    x:x_value###x_val
    }
    print(sess.run([h_r],feed_dict=feed_dict))

```

In [7]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [64, 28,28,1])

w= tf.truncated_normal([5,5,1,32], stddev=0.1)

b = tf.constant(0.1, shape=[32])

h = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME') + b

h_bn = tf.contrib.layers.batch_norm(h, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=True)#, scope="bn")

h_r = tf.nn.relu(h_bn)

x_value=tf.random_normal(shape=[64, 28,28,1],mean=5,stddev=2.0)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_val=sess.run(x_value)
    feed_dict = {
    x:x_val
    }
    print(sess.run([h_r],feed_dict=feed_dict))

    

[array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          5.26139922e-02, 9.96644516e-03, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          1.46948135e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          6.30268693e-01, 0.00000000e+00, 0.00000000e+00],
         [3.66652817e-01, 0.00000000e+00, 3.05536985e-01, ...,
          1.38459146e+00, 0.00000000e+00, 0.00000000e+00]],

        [[0.00000000e+00, 7.37867057e-02, 0.00000000e+00, ...,
          0.00000000e+00, 7.07124233e-01, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 4.50746000e-01, ...,
          0.00000000e+00, 1.44242597e+00, 0.00000000e+00],
       

----
#### tf.group和tf.identify

> tf.group是个组合操作.它返回的是op.但是它会组合一组ops.

In [2]:
import tensorflow as tf
x = tf.Variable(0.0)
x_plus = tf.assign_add(x, 1) # 这是一个引用加法.
with tf.control_dependencies([x_plus]):#只有当内部为操作时以来才会生效, 它会让这个[x_puls]先执行,执行完之后在执行其修饰的操作内容.
    #y = tf.identity(x)#将该语句变为操作
    y = x
    update = tf.group(y)#将该语句变为操作
init = tf.global_variables_initializer()
with tf.Session() as session:
    init.run()
    for i in range(5):
        session.run(update)
        print("x_plus:",x_plus.eval())
        print(y.eval())
    print(x.eval())#5


x_plus: 2.0
2.0
x_plus: 4.0
4.0
x_plus: 6.0
6.0
x_plus: 8.0
8.0
x_plus: 10.0
10.0
10.0


看下`assign_add`的功效

In [3]:
import tensorflow as tf
x=tf.Variable(0.0)
y=tf.add(x,1)
ref_x=tf.assign_add(x,1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(x.eval())
    print(y.eval())
    print(ref_x.eval())
    print("after assigned add",x.eval())

0.0
1.0
1.0
after assigned add 1.0


`tf.identify`的效果:
> 也是一种引用

In [8]:
import tensorflow as tf
x = tf.Variable(0.0)
x_plus = tf.assign_add(x, 1) # 这是一个引用加法.
with tf.control_dependencies([x_plus]):#只有当内部为操作时以来才会生效, 它会让这个[x_puls]先执行,执行完之后在执行其修饰的操作内容.
    y=tf.identity(x)
init = tf.global_variables_initializer()
with tf.Session() as session:
    init.run()
    for i in range(5):
        print(y.eval())
    print(x.eval())#5

1.0
2.0
3.0
4.0
5.0
5.0


---
#### tf.shape
---
可以取到该tensor的shape信息.

In [4]:
import tensorflow as tf
x=tf.constant(0.1,shape=[1,2,3,32])
x_shape=tf.shape(x)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(x_shape.eval())

[ 1  2  3 32]
